<a href="https://colab.research.google.com/github/Daffaaq/L-I-A-R-The-Guessing-Game/blob/main/Liar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
#!/usr/bin/env python3
# tebak_angka_chaos_no_timer.py
# Versi: Tanpa timer, skor berdasarkan percobaan tersedikit.

import random
import time
import json
import os
from datetime import datetime

SCORES_FILE = "scores.json"

# ---------- Utilities ----------
def save_score(entry):
    data = []
    if os.path.exists(SCORES_FILE):
        try:
            with open(SCORES_FILE, "r", encoding="utf-8") as f:
                data = json.load(f)
        except Exception:
            data = []
    data.append(entry)
    data = data[-200:]
    with open(SCORES_FILE, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

def print_leaderboard(top=5):
    if not os.path.exists(SCORES_FILE):
        print("Leaderboard: belum ada skor.")
        return
    try:
        with open(SCORES_FILE, "r", encoding="utf-8") as f:
            data = json.load(f)
    except Exception:
        print("Leaderboard: gagal baca file skor.")
        return

    data_sorted = sorted(data, key=lambda x: x.get("attempts", 9999))

    print("\n===== LEADERBOARD (TOP {}) =====".format(top))
    for i, d in enumerate(data_sorted[:top], 1):
        name = d.get("player", "Anon")
        score = d.get("score", 0)
        tries = d.get("attempts", "?")
        lies = d.get("lies", 0)
        dt = d.get("datetime", "?")
        print(f"{i}. {name} — Score: {score} — Percobaan: {tries} — Bohong: {lies}x — {dt}")
    print("=" * 34)

# ---------- Game config ----------
MOODS = ["HAPPY", "SASSY", "BAD_MOOD", "PLAYFUL", "SAD"]
EASTER_RESPONSES = {
    69: "EHEM… angka aesthetic detected 😳",
    0: "Bro… 0 itu bukan di range ya 😐",
    1000: "Kamu kira aku mesin ATM? 😂",
}

# ---------- Main Game ----------
def main_game(player_name="Player", mode="normal"):
    print("\n=== GAME TEBAK ANGKA — MODE:", mode.upper(), "===\n")

    # Mode config (waktu dihapus)
    if mode == "easy":
        low, high = 1, 50
        bohong_chance = 0.0
        ngambek_chance = 0.02
        hint_every = 1

    elif mode == "hard":
        low, high = 1, 100
        bohong_chance = 0.30
        ngambek_chance = 0.20
        hint_every = 3

    elif mode == "extreme":
        low, high = 1, 100
        bohong_chance = 0.40
        ngambek_chance = 0.25
        hint_every = 9999

    else:
        low, high = 1, 100
        bohong_chance = 0.20
        ngambek_chance = 0.10
        hint_every = 2

    target = random.randint(low, high)
    attempts = 0
    log_bohong = []
    log_ngambek = []
    mood = random.choice(MOODS)
    cheated = False

    print(f"Saya sudah memilih angka dari {low} sampai {high}. Tebak!")
    print("(ketik 'hint' untuk cheat, 'quit' untuk menyerah)\n")

    def hint_message(direction):
        return "Terlalu kecil! Naikkan." if direction == "up" else "Terlalu besar! Turunkan."

    def cheat_hint():
        nonlocal cheated
        cheated = True
        span = max(3, (high - low) // 6)
        return f"(CHEAT) Mungkin di antara {max(low,target-span)} - {min(high,target+span)} (atau bohong 😏)"

    pesan_ngambek = [
        "Aku ngambek 😒",
        "Capee… 😤",
        "Aku diem dulu 😑",
        "Kenapa nebaknya jauh 😭",
        "Stress aku 😫"
    ]

    # Main loop
    while True:
        raw = input("Tebakanmu: ").strip()

        if raw.lower() in ("quit", "keluar"):
            print("Kamu menyerah. Angkanya:", target)
            return

        if raw.lower() in ("hint", "cheat"):
            print(cheat_hint())
            continue

        try:
            tebakan = int(raw)
        except:
            print("Masukkan angka yang valid.")
            continue

        attempts += 1

        if tebakan in EASTER_RESPONSES:
            print(EASTER_RESPONSES[tebakan])

        # Ngambek
        if random.random() < ngambek_chance:
            log_ngambek.append(attempts)
            print(random.choice(pesan_ngambek))
            continue

        if tebakan == target:
            print(f"\n🎉 BENAR! Angkanya {target}.")
            score = compute_score(attempts)

            entry = {
                "player": player_name,
                "attempts": attempts,
                "score": score,
                "lies": len(log_bohong),
                "mode": mode,
                "datetime": datetime.now().isoformat()
            }
            save_score(entry)

            print_report(attempts, len(log_bohong), len(log_ngambek))
            print_leaderboard()
            return

        lie = random.random() < bohong_chance

        true_dir = "up" if tebakan < target else "down"

        shown_dir = true_dir if not lie else ("up" if true_dir == "down" else "down")

        if lie:
            log_bohong.append(attempts)

        print(hint_message(shown_dir))

# ---------- Scoring ----------
def compute_score(attempts):
    return max(0, 100 - (attempts - 1) * 5)

def print_report(attempts, lies, ngambek_count):
    print("\n===== STATISTIK GAME =====")
    print(f"Percobaan    : {attempts}")
    print(f"Kena Bohong  : {lies}x")
    print(f"Kena Ngambek : {ngambek_count}x")

    score = compute_score(attempts)
    print(f"Skor Akhir   : {score}/100")
    print("=========================")

# ---------- Menu ----------
def choose_mode():
    print("\nPilih mode:")
    print("1. Easy")
    print("2. Normal")
    print("3. Hard")
    print("4. Extreme")
    print("5. Exit")

    return {"1":"easy","2":"normal","3":"hard","4":"extreme","5":"exit"}.get(
        input("Pilih: "), "normal"
    )

def main():
    print("=== SELAMAT DATANG DI CHAOS GUESSING GAME ===")
    player = input("Masukkan nama: ").strip() or "Player"

    while True:
        mode = choose_mode()
        if mode == "exit":
            print("Dadah 👋")
            break
        main_game(player, mode)

        if input("Main lagi? (y/n): ").lower() != "y":
            break

if __name__ == "__main__":
    main()


=== SELAMAT DATANG DI CHAOS GUESSING GAME ===
Masukkan nama: Ache

Pilih mode:
1. Easy
2. Normal
3. Hard
4. Extreme
5. Exit
Pilih: 1

=== GAME TEBAK ANGKA — MODE: EASY ===

Saya sudah memilih angka dari 1 sampai 50. Tebak!
(ketik 'hint' untuk cheat, 'quit' untuk menyerah)

Tebakanmu: 20
Terlalu kecil! Naikkan.
Tebakanmu: 25

🎉 BENAR! Angkanya 25.

===== STATISTIK GAME =====
Percobaan    : 2
Kena Bohong  : 0x
Kena Ngambek : 0x
Skor Akhir   : 95/100

===== LEADERBOARD (TOP 5) =====
1. Ache — Score: 95 — Percobaan: 2 — Bohong: 0x — 2025-12-05T09:08:23.675565
Main lagi? (y/n): y

Pilih mode:
1. Easy
2. Normal
3. Hard
4. Extreme
5. Exit
Pilih: 2

=== GAME TEBAK ANGKA — MODE: NORMAL ===

Saya sudah memilih angka dari 1 sampai 100. Tebak!
(ketik 'hint' untuk cheat, 'quit' untuk menyerah)

Tebakanmu: 90
Terlalu kecil! Naikkan.
Tebakanmu: 95
Terlalu besar! Turunkan.
Tebakanmu: 92
Terlalu kecil! Naikkan.
Tebakanmu: 93
Terlalu besar! Turunkan.
Tebakanmu: 94
Terlalu kecil! Naikkan.
Tebakanmu: 95
T